In [18]:
from scipy.io import loadmat
from os import listdir
from json import dump
import numpy as np

In [ ]:
def make_json_serializable(obj):
    if isinstance(obj, dict):
        return {k: make_json_serializable(v) for k, v in obj.items()}
    if isinstance(obj, np.ndarray):
        return make_json_serializable(obj.tolist())
    if isinstance(obj, (list, tuple)):
        return [make_json_serializable(x) for x in obj]
    if isinstance(obj, (np.integer,)):
        return int(obj)
    if isinstance(obj, (np.floating,)):
        return float(obj)
    if isinstance(obj, np.bool_):
        return bool(obj)
    if isinstance(obj, np.generic):
        return obj.item()
    return obj

full_dict = {}

for checkup_name in sorted(listdir("CU_Dynamic")):
	if checkup_name == "failed and incomplete":
		continue
	full_dict[checkup_name] = {}
	for cell in listdir(f"CU_Dynamic/{checkup_name}"):
		if cell.startswith("BW-VTC-458"):
			data = loadmat(
				f"CU_Dynamic/{checkup_name}/{cell}",
				squeeze_me=True,
				struct_as_record=False
			)["Dataset"].__dict__

			for key in data.keys():
				if key in ["_fieldnames", "Info"]:
					continue
				full_dict[checkup_name][key] = data[key]
			break

with open("full_data.json", "w") as f:
	dump(make_json_serializable(full_dict), f)